# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [53]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [54]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [55]:
# confirm that our new database was created
databases = mongo.list_database_names()
print("Databases:", databases)

Databases: ['Resources', 'admin', 'config', 'epa', 'fruits_db', 'local', 'met', 'uk_food']


In [56]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [57]:
# List the collections in the uk_food database
collections = db.list_collection_names()
print("Collections:", collections)

Collections: ['establishments']


In [58]:
# Find and display one document in the establishments collection
establishments_collection = db['establishments']
document = establishments_collection.find_one()
print("One document in establishments collection:")
pprint(document)

One document in establishments collection:
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('662b24a75e7d126ef567494c'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [59]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [60]:
# Insert the new restaurant into the collection
insert_result = establishments_collection.insert_one(new_restaurant)
print("Inserted new restaurant:", insert_result.inserted_id)

Inserted new restaurant: 662b29ae5e7d126ef5674952


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [61]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
restaurant_type = establishments_collection.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessTypeID": 1, "_id": 0})
print("BusinessTypeID for 'Restaurant/Cafe/Canteen':", restaurant_type)


BusinessTypeID for 'Restaurant/Cafe/Canteen': {'BusinessTypeID': ''}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [62]:
# Update the new restaurant with the correct BusinessTypeID
update_result = establishments_collection.update_one({"BusinessName": "Penang Flavours"}, {"$set": {"BusinessTypeID": restaurant_type["BusinessTypeID"]}})
print("Updated restaurant:", update_result.modified_count)

Updated restaurant: 0


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [64]:
# Find how many documents have LocalAuthorityName as "Dover"
count_dover = establishments_collection.count_documents({"LocalAuthorityName": "Dover"})
print("Number of documents with LocalAuthorityName as 'Dover':", count_dover)

Number of documents with LocalAuthorityName as 'Dover': 0


In [65]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments_collection.delete_many({"LocalAuthorityName": "Dover"})
print("Number of documents deleted:", delete_result.deleted_count)

Number of documents deleted: 0


In [66]:
# Check if any remaining documents include Dover
remaining_documents = establishments_collection.find({"LocalAuthorityName": {"$regex": ".*Dover.*"}})

# Check if any documents were found
if remaining_documents.count() > 0:
    print("There are remaining documents with LocalAuthorityName including 'Dover'.")
else:
    print("There are no remaining documents with LocalAuthorityName including 'Dover'.")

AttributeError: 'Cursor' object has no attribute 'count'

In [67]:
# Check that other documents remain with 'find_one'
# Find one document after deleting documents with LocalAuthorityName as "Dover"
remaining_document = establishments_collection.find_one()

# Check if a document was found
if remaining_document:
    print("At least one document remains in the collection.")
else:
    print("No documents remain in the collection.")

At least one document remains in the collection.


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [68]:
# Change the data type from String to Decimal for longitude and latitude
from decimal import Decimal

# Update the data type for longitude and latitude fields
update_result = establishments_collection.update_many({}, [
    {"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}},
    {"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}
])

print("Number of documents updated:", update_result.modified_count)

Number of documents updated: 1


Use `update_many` to convert `RatingValue` to integer numbers.

In [69]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
update_result = establishments_collection.update_many(
    {"RatingValue": {"$in": non_ratings}},
    {"$set": {"RatingValue": None}}
)

print("Number of documents updated with null RatingValue:", update_result.modified_count)


Number of documents updated with null RatingValue: 0


In [70]:
# Change the data type from String to Integer for RatingValue
update_result = establishments_collection.update_many(
    {"RatingValue": {"$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

print("Number of documents updated with integer RatingValue:", update_result.modified_count)

Number of documents updated with integer RatingValue: 0


In [71]:
# Check that the coordinates and rating value are now numbers

